In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# Define the damped pendulum model.
def damped_pendulum_model(t, A, T, phi, offset, gamma):
    """
    Damped pendulum displacement model.
    
    Parameters:
        t : array_like
            Time values.
        A : float
            Initial amplitude.
        T : float
            Period.
        phi : float
            Phase offset.
        offset : float
            Vertical offset.
        gamma : float
            Damping coefficient.
            
    Returns:
        x : array_like
            Displacement at time t.
    """
    return A * np.exp(-gamma * t) * np.cos(2 * np.pi * t / T + phi) + offset

# -------------------------------
# Read the CSV data.
# -------------------------------
# Replace 'data.csv' with the path to your CSV file.
data = pd.read_csv('data.csv')

# Assumes your CSV has headers 't' and 'x'
t_data = data['t'].values
x_data = data['x'].values

# If you have uncertainties per point, you can load them as well.
# Otherwise, we'll assume a constant error for all data points:
sigma_data = np.full_like(t_data, 0.05)  # adjust as needed

# -------------------------------
# Initial parameter guesses:
# -------------------------------
# Estimate amplitude from data range, a guess for period (e.g., 1 s), zero phase,
# offset as the mean of x_data, and a small damping coefficient.
p0 = [
    (np.max(x_data) - np.min(x_data)) / 2,  # A
    1.0,                                   # T (s)
    0.0,                                   # phi
    np.mean(x_data),                       # offset
    0.1                                    # gamma (s^-1)
]

# -------------------------------
# Perform the chi² fit using curve_fit.
# -------------------------------
popt, pcov = curve_fit(damped_pendulum_model, t_data, x_data, sigma=sigma_data,
                       absolute_sigma=True, p0=p0)

# Extract best-fit parameters and their uncertainties.
A_fit, T_fit, phi_fit, offset_fit, gamma_fit = popt
A_err, T_err, phi_err, offset_err, gamma_err = np.sqrt(np.diag(pcov))

print(f"Fitted period T = {T_fit:.4f} ± {T_err:.4f} s")
print(f"Fitted damping gamma = {gamma_fit:.4f} ± {gamma_err:.4f} s⁻¹")

# -------------------------------
# Calculate gravitational constant g:
# -------------------------------
# For a simple pendulum: T = 2π √(L/g)  =>  g = 4π² L / T²
L = 1.0  # Pendulum length in meters (update with your actual length)
g = 4 * np.pi**2 * L / T_fit**2

# Error propagation: dg/dT = -8π² L / T³, so:
g_err = 8 * np.pi**2 * L * T_err / T_fit**3

print(f"Calculated gravitational constant g = {g:.4f} ± {g_err:.4f} m/s²")

# -------------------------------
# (Optional) Plot the data and the fitted model:
# -------------------------------
t_fit = np.linspace(np.min(t_data), np.max(t_data), 1000)
x_fit = damped_pendulum_model(t_fit, *popt)

plt.errorbar(t_data, x_data, yerr=sigma_data, fmt='o', label='Data')
plt.plot(t_fit, x_fit, label='Damped Fit', color='red')
plt.xlabel("Time (s)")
plt.ylabel("x (m)")
plt.legend()
plt.show()